<a href="https://colab.research.google.com/github/abinayalakshmi852-spec/customer-support-ticket-triage/blob/main/Week1_Customer_Support_Triage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
from datetime import timedelta

# ===============================
# Configuration (Correct File Path)
# ===============================
FILE_PATH = '/content/customer_support_tickets.csv'  # Make sure the file is uploaded to Colab

# ===============================
# 1. Load and Explore Dataset
# ===============================
df = pd.read_csv(FILE_PATH)

print(f"Dataset Loaded: {df.shape[0]} rows, {df.shape[1]} columns")
print("\nMissing values per column:\n", df.isnull().sum())

# ===============================
# 2. Clean Ticket Descriptions
# ===============================
df['Cleaned_Description'] = (
    df['Ticket Description']
    .astype(str)
    .str.lower()
    .replace(r'[^a-z0-9\s]', '', regex=True)
    .replace(r'\s+', ' ', regex=True)
    .str.strip()
)

# ===============================
# 3. Issue Classification (Rule-Based)
# ===============================
def classify_issue(text):
    if any(k in text for k in ['pay', 'billing', 'charge', 'invoice', 'card']):
        return 'PAYMENT'
    if any(k in text for k in ['login', 'password', 'account', 'access', 'sign']):
        return 'LOGIN'
    if any(k in text for k in ['shipping', 'delivery', 'order', 'track']):
        return 'DELIVERY'
    if any(k in text for k in ['refund', 'money back', 'return']):
        return 'REFUND'
    if any(k in text for k in ['bug', 'error', 'crash', 'glitch', 'not working']):
        return 'BUG'
    return 'GENERAL'

df['Issue_Category'] = df['Cleaned_Description'].apply(classify_issue)

# ===============================
# 4. Priority Assignment
# ===============================
def assign_priority(text):
    if any(k in text for k in ['urgent', 'critical', 'emergency', 'asap', 'blocking']):
        return 'P0'
    if any(k in text for k in ['high', 'broken', 'failure']):
        return 'P1'
    if any(k in text for k in ['medium', 'issue', 'problem']):
        return 'P2'
    return 'P3'

df['Priority_Level'] = df['Cleaned_Description'].apply(assign_priority)

# ===============================
# 5. SLA & Due Time Calculation
# ===============================
df['First Response Time'] = pd.to_datetime(df['First Response Time'], errors='coerce')

# Assume ticket created 1 hour before first response
df['Ticket_Created_At'] = df['First Response Time'].fillna(pd.Timestamp.now()) - pd.Timedelta(hours=1)

sla_mapping = {'P0': 2, 'P1': 4, 'P2': 8, 'P3': 24}
df['SLA_Hours'] = df['Priority_Level'].map(sla_mapping)

df['Due_Time'] = df['Ticket_Created_At'] + pd.to_timedelta(df['SLA_Hours'], unit='h')

# ===============================
# 6. Support Manager Summary Report
# ===============================
print("\n--- Support Manager Report ---")
print("\nIssues by Category:\n", df['Issue_Category'].value_counts())
print("\nIssues by Priority:\n", df['Priority_Level'].value_counts())

if 'Customer Satisfaction Rating' in df.columns:
    print(f"\nAverage Satisfaction Rating: {df['Customer Satisfaction Rating'].mean():.2f}")

# ===============================
# 7. Export Final Report
# ===============================
OUTPUT_FILE = '/content/final_support_report.csv'
df.to_csv(OUTPUT_FILE, index=False)

print(f"\nFinal output saved to: {OUTPUT_FILE}")

Dataset Loaded: 8469 rows, 17 columns

Missing values per column:
 Ticket ID                          0
Customer Name                      0
Customer Email                     0
Customer Age                       0
Customer Gender                    0
Product Purchased                  0
Date of Purchase                   0
Ticket Type                        0
Ticket Subject                     0
Ticket Description                 0
Ticket Status                      0
Resolution                      5700
Ticket Priority                    0
Ticket Channel                     0
First Response Time             2819
Time to Resolution              5700
Customer Satisfaction Rating    5700
dtype: int64

--- Support Manager Report ---

Issues by Category:
 Issue_Category
GENERAL     5288
BUG         1152
LOGIN        783
PAYMENT      600
DELIVERY     459
REFUND       187
Name: count, dtype: int64

Issues by Priority:
 Priority_Level
P2    7928
P3     300
P0     184
P1      57
Name: count, 